In [43]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_addons as tfa

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import pandas as pd

## 数据处理

转换成我们想要的train, val, test数据格式

In [2]:
X = np.load("/home/blu/workspace/GCN/STGCN-PyTorch/data/node_values.npy").transpose((1, 2, 0))
X = X.astype(np.float32)

In [3]:
X.shape

(207, 2, 34272)

In [4]:
X[:,0,:].shape

(207, 34272)

In [5]:
a = X[:,0,:].T

In [6]:
df = pd.DataFrame(a)
print(df.shape)
df.head()

(34272, 207)


,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125,67.125,59.625000,62.750000,...,45.625000,65.500,64.500000,66.428574,66.875,59.375000,69.000000,59.250000,69.000000,61.875
1,62.666668,68.555557,65.444443,62.444443,64.444443,68.111115,65.000,65.000,57.444443,63.333332,...,50.666668,69.875,66.666664,58.555557,62.000,61.111111,64.444443,55.888889,68.444443,62.875
2,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500,64.250,63.875000,65.375000,...,44.125000,69.000,56.500000,59.250000,68.125,62.500000,65.625000,61.375000,69.857140,62.000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,...,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000


In [7]:
x = []
y = []
x_offset = 12
y_offset = 1

In [8]:
index_max = (df.shape[0]) - x_offset - y_offset
for i in tqdm(range(index_max)):
    x_array = df[i : i + x_offset].values
    y_array = df[i + x_offset : i + x_offset + y_offset].values
    x.append(x_array.tolist())
    y.append(y_array.tolist())
x = np.stack(x, axis=0)    
y = np.stack(y, axis=0)

100%|██████████| 34259/34259 [00:14<00:00, 2374.73it/s]


In [12]:
print("x.shape = ", x.shape, " y.shape = ", y.shape)
dataset = np.concatenate((x, y), axis=1)

x.shape =  (34259, 12, 207)  y.shape =  (34259, 1, 207)


In [27]:
dataset.shape

(34259, 13, 207)

In [45]:
dataset_shuffle = np.random.permutation(dataset)
print(dataset_shuffle.shape)

(34259, 13, 207)


In [50]:
x = dataset_shuffle[:,0:12,:]
x.shape

(34259, 12, 207)

In [60]:
y = dataset_shuffle[:,-1,:]
y.shape

(34259, 207)

In [55]:
x_uni = (x - x.mean())/x.std()

In [61]:
num_samples = x.shape[0]
num_test = round(num_samples * 0.2)
num_train = round(num_samples * 0.7)
num_val = num_samples - num_test - num_train

x_train, y_train = x_uni[:num_train], y[:num_train]
x_val, y_val = (
        x_uni[num_train: num_train + num_val],
        y[num_train: num_train + num_val],
    )
x_test, y_test = x_uni[-num_test:], y[-num_test:]

In [62]:
np.save("/home/blu/workspace/graduate_project/STGCN/METR-LA/x_train.npy", x_train)
np.save("/home/blu/workspace/graduate_project/STGCN/METR-LA/y_train.npy", y_train)
np.save("/home/blu/workspace/graduate_project/STGCN/METR-LA/x_val.npy", x_val)
np.save("/home/blu/workspace/graduate_project/STGCN/METR-LA/y_val.npy", y_val)
np.save("/home/blu/workspace/graduate_project/STGCN/METR-LA/x_test.npy", x_test)
np.save("/home/blu/workspace/graduate_project/STGCN/METR-LA/y_test.npy", y_test)

## Seq2Seq Test

In [63]:
print("INFO: x_train:", x_train.shape, "y_train:", y_train.shape)
print("INFO: x_val:", x_val.shape, "y_val:", y_val.shape)
print("INFO: x_test:", x_test.shape, "y_test:", y_test.shape)

INFO: x_train: (23981, 12, 207) y_train: (23981, 207)
INFO: x_val: (3426, 12, 207) y_val: (3426, 207)
INFO: x_test: (6852, 12, 207) y_test: (6852, 207)


In [105]:
input_test = x_train[0:3][:,11:12,:]
print(input_test.shape)

(3, 1, 207)


In [114]:
from tensorflow.keras import layers


class Linear(layers.Layer):

  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                              dtype='float32'),
                         trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                              dtype='float32'),
                         trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

encoder_emb_inp = tf.keras.Input(shape=(2, 2), name='Input')
linear_layer = Linear(4, 2)
y = linear_layer(encoder_emb_inp)

model = tf.keras.Model(inputs = encoder_emb_inp, outputs = y, name = "test")
model.summary()

Model: "test"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 2, 2)]            0         
_________________________________________________________________
linear_2 (Linear)            (None, 2, 4)              12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [131]:
#Basic GCN
A = np.load("/home/blu/workspace/GCN/STGCN-PyTorch/data/adj_mat.npy")

class GCN(tf.keras.layers.Layer):
  def __init__(self, units=207, input_dim=207, matrix = None):
    super(GCN, self).__init__()
    w_init = tf.random_normal_initializer()
    self.w1 = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                              dtype='float32'), trainable=True)
    self.w2 = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                              dtype='float32'), trainable=True)
    self.w3 = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                              dtype='float32'), trainable=True)
    self.w4 = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                              dtype='float32'), trainable=True)
    b_init = tf.zeros_initializer()
    self.b1 = tf.Variable(initial_value=b_init(shape=(1,),
                                              dtype='float32'), trainable=True)
    self.b2 = tf.Variable(initial_value=b_init(shape=(1,),
                                              dtype='float32'), trainable=True)
    self.b3 = tf.Variable(initial_value=b_init(shape=(1,),
                                              dtype='float32'), trainable=True)
    self.matrix = tf.convert_to_tensor(matrix)

  def call(self, inputs):
    weight_1 = tf.multiply(self.matrix,self.w1)
    weight_2 = tf.multiply(self.matrix,self.w2)
    weight_3 = tf.multiply(self.matrix,self.w3)
    weight_4 = tf.multiply(self.matrix,self.w4)
    inputs = tf.reshape(inputs, [-1, 207])
    y1 = tf.nn.relu(tf.matmul(inputs, weight_1))
    y2 = tf.nn.relu(tf.matmul(y1, weight_2) + self.b1)
    y3 = tf.nn.relu(tf.matmul(y2, weight_3) + self.b2)
    y4 = tf.matmul(y3, weight_4) + self.b3
    return y3

graph_inp = tf.keras.Input(shape=(1,207), name='graph_input')
gcn_layer = GCN(207, 207, A)
gcn_output = gcn_layer(graph_inp)

model = tf.keras.Model(inputs = graph_inp, outputs = gcn_output, name = "gcn")
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(), 
              metrics=[tf.keras.metrics.MeanSquaredError(), 
                       tf.keras.metrics.MeanAbsolutePercentageError(), 
                       tf.keras.metrics.MeanAbsoluteError(), 
                       tf.keras.metrics.RootMeanSquaredError()])
print("INFO: Start training...")

history = model.fit(x_train[:,-2:-1,:], 
                    y_train,
                    batch_size=512,
                    epochs=500,
                    validation_data=(x_val[:,-2:-1,:],y_val))

print("INFO: Start testing...")
dataset = tf.data.Dataset.from_tensor_slices((x_test[:,-2:-1,:], y_test))
dataset = dataset.batch(512)
model.evaluate(dataset)

Model: "gcn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
graph_input (InputLayer)     [(None, 1, 207)]          0         
_________________________________________________________________
gcn_35 (GCN)                 (None, 207)               171399    
Total params: 171,399
Trainable params: 171,399
Non-trainable params: 0
_________________________________________________________________
INFO: Start training...
Train on 23981 samples, validate on 3426 samples
Epoch 1/500
23981/23981 [==============================] - 1s 42us/sample - loss: 3296.8070 - mean_squared_error: 3296.8064 - mean_absolute_percentage_error: 2348087.5000 - mean_absolute_error: 53.7403 - root_mean_squared_error: 57.4178 - val_loss: 3252.8362 - val_mean_squared_error: 3252.8362 - val_mean_absolute_percentage_error: 6163336.5000 - val_mean_absolute_error: 53.0958 - val_root_mean_squared_error: 57.0336
Epoch 2/500
23981/23981 [=

[77.79038756234306, 79.0439, 964743600.0, 4.792851, 8.890663]

In [64]:
#Basic LSTM
encoder_emb_inp = tf.keras.Input(shape=(12, 207), name='Input')
lstm1,state_h,state_c = tf.keras.layers.LSTM(256, return_sequences=False, return_state=True)(encoder_emb_inp)
lstm2 = tf.keras.layers.LSTM(256, return_sequences=False)(lstm1[:,np.newaxis,:])
lstm2_out = tf.keras.layers.Dense(units=207, activation=tf.nn.relu)(lstm2)
model = tf.keras.Model(inputs = encoder_emb_inp, outputs = lstm2_out, name = "seq2seq")
# y = model.predict(x_train[0:3])
# y.shape
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(), 
              metrics=[tf.keras.metrics.MeanSquaredError(), 
                       tf.keras.metrics.MeanAbsolutePercentageError(), 
                       tf.keras.metrics.MeanAbsoluteError(), 
                       tf.keras.metrics.RootMeanSquaredError()])

Model: "seq2seq"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 12, 207)]         0         
_________________________________________________________________
lstm_2 (LSTM)                [(None, 256), (None, 256) 475136    
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 1, 256)]          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 207)               53199     
Total params: 1,053,647
Trainable params: 1,053,647
Non-trainable params: 0
_________________________________________________________________


In [67]:
# Basic GRU
encoder_emb_inp = tf.keras.Input(shape=(12, 207), name='Input')
gru1 = tf.keras.layers.GRU(256, return_sequences=False, return_state=False, activation='tanh')(encoder_emb_inp)
gru2 = tf.keras.layers.GRU(256, return_sequences=False, return_state=False, activation='relu')(gru1[:,np.newaxis,:])
gru2_out = tf.keras.layers.Dense(units=207, activation=tf.nn.relu)(gru2)
model = tf.keras.Model(inputs = encoder_emb_inp, outputs = gru2_out, name = "gru")
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError(), 
              metrics=[tf.keras.metrics.MeanSquaredError(), 
                       tf.keras.metrics.MeanAbsolutePercentageError(), 
                       tf.keras.metrics.MeanAbsoluteError(), 
                       tf.keras.metrics.RootMeanSquaredError()])

Model: "gru"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 12, 207)]         0         
_________________________________________________________________
gru (GRU)                    (None, 256)               357120    
_________________________________________________________________
tf_op_layer_strided_slice_2  [(None, 1, 256)]          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               394752    
_________________________________________________________________
dense_2 (Dense)              (None, 207)               53199     
Total params: 805,071
Trainable params: 805,071
Non-trainable params: 0
_________________________________________________________________


In [68]:
print("INFO: Start training...")

history = model.fit(x_train, 
                    y_train,
                    batch_size=512,
                    epochs=200,
                    validation_data=(x_val,y_val))

print("INFO: Start testing...")
dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
dataset = dataset.batch(512)
model.evaluate(dataset)

model.save('/home/blu/workspace/graduate_project/STGCN/METR-LA/seq2seq_gru.h5')

INFO: Start training...
Train on 23981 samples, validate on 3426 samples
Epoch 1/200
23981/23981 [==============================] - 8s 335us/sample - loss: 1574.7361 - mean_squared_error: 1574.7362 - mean_absolute_percentage_error: 729552832.0000 - mean_absolute_error: 31.9396 - root_mean_squared_error: 39.6829 - val_loss: 254.4503 - val_mean_squared_error: 254.4503 - val_mean_absolute_percentage_error: 1345123456.0000 - val_mean_absolute_error: 8.9712 - val_root_mean_squared_error: 15.9515
Epoch 2/200
23981/23981 [==============================] - 5s 192us/sample - loss: 213.0105 - mean_squared_error: 213.0105 - mean_absolute_percentage_error: 1279928960.0000 - mean_absolute_error: 9.3106 - root_mean_squared_error: 14.5949 - val_loss: 194.6053 - val_mean_squared_error: 194.6053 - val_mean_absolute_percentage_error: 1367377664.0000 - val_mean_absolute_error: 8.7447 - val_root_mean_squared_error: 13.9501
Epoch 3/200
23981/23981 [==============================] - 5s 192us/sample - loss: 